# Data extraction

Install packages

In [1]:
# PACKAGES
import time
import csv

# standard processing packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

# hugging face packages
from datasets import load_dataset

/Users/chloehashimoto/miniconda3/envs/SNLP00/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Read in the words to filter out

In [2]:
with open('Keywords List - Short Version.csv', 'r') as f:
    reader = csv.reader(f)
    words_short = list(reader)
    
words_short
#TODO - turn this into the long version later

[['Anthropocene',
  'Carbon footprint',
  'Carbon-neutral',
  'Renewable Energy',
  'Greenwashing',
  'Zero-carbon',
  'Climate crisis',
  'Corporate Social Responsibility',
  'Paris Agreement',
  'Paris Climate Agreement',
  'Sea Level Rising',
  'Ocean Level Rising',
  'Carbon Capture and Storage',
  'Intergovernmental Panel on Climate Change (IPCC)',
  'Environmental Justice',
  'Anthropogenic emissions',
  'Climate Threshold',
  'Extreme Weather Event',
  'Carbon Sequestration',
  'Enhanced Greenhouse Effect',
  'Global Warming Potential',
  'Ecological Change',
  'Sea-Surface Temperature',
  'Climate Policy',
  'Fossil Fuels',
  'Nature Conservation']]

In [3]:
with open('Keywords List - Long Version.csv', 'r') as f:
    reader = csv.reader(f)
    words_long = list(reader)
    
words_long
#TODO - turn this into the long version later

[['warming',
  'temperature',
  'environment',
  'global warming',
  'climate change',
  'precipitation',
  'humidity',
  'environmental',
  'greenhouse gas',
  'biodiversity',
  'atmospheric',
  'meteorology',
  'rainfall',
  'biosphere',
  'landscape',
  'oceans',
  'economic',
  'pollution',
  'köppen climate classification',
  'energy',
  'wind',
  'arctic',
  'climatic',
  'tropical',
  'ecological',
  'climate',
  'atmospheric pressure',
  'temperate',
  'anthropogenic',
  'carbon emissions',
  'biome',
  'carbon emission',
  'hydrosphere',
  'paleoclimatology',
  'carbon sequestration',
  'ecosystem',
  'enviroment',
  'extinction',
  'meteorological',
  'ocean current',
  'climatologists',
  'reducing carbon emissions',
  'permafrost',
  'atmospheric carbon dioxide',
  'world meteorological organization',
  'subarctic',
  'sustainability',
  'marine ecosystems',
  'warmer temperatures',
  'greenhouse emissions',
  'carbon dioxide emissions',
  'climatologist',
  'climatic condi

Pull out the data from hugging face

In [4]:
def pull_data(dataset_name = "cerebras/SlimPajama-627B", n = 5499):
    """
    Args:
        dataset_name (string): name of the dataset to pull from
        n (int): number of samples to pull from the dataset (defaults to 5499 which is the maximum number of files)
        
    Returns:
        dataset (datasets.dataset_dict.DatasetDict): dictionary of datasets subsplits from huggingface
    
    """
    data_files = {str(i): f"test/chunk1/example_holdout_{i}.jsonl.zst" for i in range(n)}
    dataset = load_dataset(dataset_name, data_files=data_files) # load test dataset
    
    return dataset

In [25]:
start_pull = time.time()
datafiles_max = pull_data(n = 5499) # pull the files
end_pull = time.time()

print(f"Files pulled in {end_pull - start_pull} seconds.")

Retrying in 1s [Retry 1/5].
Generating 0 split: 16 examples [00:00, 645.84 examples/s]
Generating 1 split: 16 examples [00:00, 11518.86 examples/s]
Generating 2 split: 16 examples [00:00, 13902.81 examples/s]
Generating 3 split: 16 examples [00:00, 17058.68 examples/s]
Generating 4 split: 16 examples [00:00, 12499.32 examples/s]
Generating 5 split: 16 examples [00:00, 15577.73 examples/s]
Generating 6 split: 16 examples [00:00, 13617.87 examples/s]
Generating 7 split: 16 examples [00:00, 16100.98 examples/s]
Generating 8 split: 16 examples [00:00, 15966.90 examples/s]
Generating 9 split: 16 examples [00:00, 15595.83 examples/s]
Generating 10 split: 16 examples [00:00, 16336.14 examples/s]
Generating 11 split: 16 examples [00:00, 18285.79 examples/s]
Generating 12 split: 16 examples [00:00, 14611.12 examples/s]
Generating 13 split: 16 examples [00:00, 17185.37 examples/s]
Generating 14 split: 16 examples [00:00, 19536.79 examples/s]
Generating 15 split: 16 examples [00:00, 13640.01 exam

Files pulled in 5856.359065771103 seconds.


In [26]:
# save the datafiles to json file

# Initialize an empty list to hold the dataframes
dfs = []

# Loop over the datasets in the DatasetDict
for key in datafiles_max:
    df = datafiles_max[key].to_pandas()
    dfs.append(df)

# Concatenate all the dataframes
combined_df = pd.concat(dfs, ignore_index=True)

# Write the combined dataframe to a JSON file
combined_df.to_json("datafiles_max.json", orient='records', lines=True)

Filter the contents for related words - SHORT version

In [14]:
# function to filter out words in a dataset
def filter_dict(dataset, words):
    """
    Args:
        dataset (datasets.dataset_dict.DatasetDict, datasets.arrow_dataset.Dataset): dictionary of datasets subsplits from huggingface
        words (list): list of words to filter the dataset for
    """
    
    filtered_dataset = dataset.filter(lambda x: any(word in x['text'] for word in words)) # filter for any of the words in the list
    
    return filtered_dataset

In [27]:
start_filter = time.time()
filtered_these_short = filter_dict(datafiles_max, words_short[0]) # filter the files
end_filter = time.time()

print(f"Files filtered in {end_filter - start_filter}.")

Filter: 100%|██████████| 16/16 [00:00<00:00, 49.11 examples/s] 


Files filtered in 6747.561989784241.


Filtering - LONG version

In [28]:
start_filter = time.time()
filtered_these_long = filter_dict(datafiles_max, words_long[0]) # filter the files
end_filter = time.time()

print(f"Files filtered in {end_filter - start_filter}.")

Filter: 100%|██████████| 16/16 [00:00<00:00, 88.59 examples/s]


Files filtered in 161508.25065684319.


Save to JSON - short

In [29]:
# Initialize an empty list to hold the dataframes
dfs_short = []

# Loop over the slices
for i in range(5499):  # replace with the number of slices you have
    df = filtered_these_short[str(i)].to_pandas()
    dfs_short.append(df)

# Concatenate all the dataframes
combined_df_short = pd.concat(dfs_short, ignore_index=True)
combined_df_short.head()

# Write the combined dataframe to a JSON file
combined_df_short.to_json("combined_max_short.json", orient='records', lines=True)

Save to JSON - long

In [ ]:
# Initialize an empty list to hold the dataframes
dfs_long = []

# Loop over the slices
for i in range(5499):  # replace with the number of slices you have
    df = filtered_these_long[str(i)].to_pandas()
    dfs_long.append(df)

# Concatenate all the dataframes
combined_df_long = pd.concat(dfs_long, ignore_index=True)
combined_df_long.head()

# Write the combined dataframe to a JSON file
combined_df_long.to_json("combined_max_long.json", orient='records', lines=True)

Pulled out 1000 first files in 10 minutes.

Filtered out the short list in 28 seconds and the long list in 58 seconds.

In [5]:
df = pd.read_json("combined_max_short.json", orient='records', lines=True)

# Display the first few rows of the dataframe
df.shape

(213, 2)